# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse import hstack

# Read in data

In [2]:
games = pd.read_csv('data/ready_to_model.csv')
games.head(2)

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description,model_description,clean_genres,clean_descrip
0,Mass Effect 2,mass-effect-2,94.0,8.9,2010-01-26,The Mass Effect trilogy is a science fiction a...,"{'Action RPG', 'Role-Playing'}",M,4806.0,"Mass Effect II is a sequel to Mass Effect one,...","Mass Effect II is a sequel to Mass Effect one,...",action role-playing rpg,mass effect ii sequel mass effect one followin...
1,Starcraft II: Wings of Liberty,starcraft-ii-wings-of-liberty,93.0,8.3,2010-07-27,StarCraft II continues the epic saga of the Pr...,"{'Strategy', 'Command', 'Sci-Fi', 'Real-Time'}",T,38067.0,StarCraft II: Wings of Liberty is a strategy g...,StarCraft II continues the epic saga of the Pr...,sci-fi strategy command real-time,starcraft ii continues epic saga protoss terra...


# Building the recommender

- Use CountVectorizer on the genres
- Use TFIDFVectorizer on the game descriptions

In [3]:
# CountVectorizer
cvec = CountVectorizer()
cvec_matrix = cvec.fit_transform(games['clean_genres'])

# TFIDFVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 3),
                        min_df=5,
                        max_df=0.75)

tfidf_matrix = tfidf.fit_transform(games['clean_descrip'])

In [4]:
# Combine the two matrices
combined_matrices = hstack((tfidf_matrix, cvec_matrix))

In [5]:
# Calculate distance and create dataframe from results
recommender = pairwise_distances(combined_matrices, metric='cosine')

rec_df = pd.DataFrame(recommender,
                      index=games['name'],
                      columns=games['name'])

In [6]:
rec_df.head(3)

name,Mass Effect 2,Starcraft II: Wings of Liberty,World of Warcraft: Cataclysm,Sid Meier's Civilization V,BioShock 2,Super Meat Boy,Battlefield: Bad Company 2,Monkey Island 2 Special Edition: LeChuck's Revenge,Borderlands: The Secret Armory of General Knoxx,Need for Speed: Hot Pursuit,...,Warlander,Romance of the Three Kingdoms XIV,Orangeblood,Praetorians HD Remaster,Table Manners,Minecraft: Story Mode - A Telltale Games Series - The Complete Edition,Minecraft: Story Mode - Season Two: The Telltale Series,Sam & Max: The Devil's Playhouse,Cognition: An Erica Reed Thriller,Back to the Future: The Game
name,,,,,,,,,,,,,,,,,,,,,
Mass Effect 2,0.000000,0.992625,0.657152,0.995352,0.838656,0.772796,0.828137,0.995116,0.839878,0.997196,...,0.189530,0.994758,0.547795,0.997379,0.997518,0.993505,0.997011,0.999199,0.997812,0.857789
Starcraft II: Wings of Liberty,0.992625,0.000000,0.989835,0.832916,0.728561,0.993466,0.993973,0.988788,0.729537,0.995818,...,0.990679,0.823659,0.993797,0.477485,0.993621,0.992017,0.992211,0.997134,0.994390,0.758814
World of Warcraft: Cataclysm,0.657152,0.989835,0.000000,0.995820,0.993670,0.992406,0.996737,0.988277,0.983828,0.994481,...,0.654121,0.993460,0.614823,0.996675,0.995718,0.996392,0.994097,0.996586,0.992467,0.998116


# Evaluate recommender

## Rocket League

Rocket League is essentially a car soccer-like game. You control a car manuevering it to push a ball into the opposing team's goal. The top 10 recommendations returned are mostly racing games and a few soccer games, which shows that the recommender does a pretty good job selecting games that have similar descriptions and genres.

Source: https://en.wikipedia.org/wiki/Rocket_League

In [7]:
rec_df['Rocket League'].sort_values()[1:11]

name
Formula Fusion               0.414325
Antigraviator                0.418578
Trailblazers                 0.426053
Obliteracers                 0.428721
Desert Child                 0.429712
Frozen Cortex                0.475341
Speedball 2 HD               0.476041
Pro Evolution Soccer 2016    0.481054
Redout (2016)                0.484130
Pro Evolution Soccer 2019    0.484935
Name: Rocket League, dtype: float64

## The Wolf Among Us

The Wolf Among Us is a point-and-click, episodic game developed by Telltale Games based on a comic about fairy tale characters. You play as Bigby Wolf aka The Big Bad Wolf, trying to solve a string of murders in Fabletown. Looking at the recommendations, it makes sense that the recommended games are also either games developed by Telltale Games, episodic games, point-and-click type games, or a combination of those.

In [8]:
rec_df['The Wolf Among Us'].sort_values()[1:11]

name
The Walking Dead: The Telltale Series - A New Frontier    0.176030
The Walking Dead: Season Two - A Telltale Games Series    0.178198
Dead Synchronicity: Tomorrow Comes Today                  0.188719
Game of Thrones: A Telltale Games Series                  0.188825
Shadowgate                                                0.195279
The Bunker                                                0.195900
The Dream Machine                                         0.195985
Cognition: An Erica Reed Thriller                         0.265694
Edna & Harvey: Harvey's New Eyes                          0.265914
Gomo                                                      0.266757
Name: The Wolf Among Us, dtype: float64

# Limitations

Game recommdendations are limited to the number of similar games in the data. Since Overcooked! is the only cooking game in the data, we get a list of recommendations that are not similar to Overcooked! at all in terms of description but only in genre (simulation games). 

In [9]:
rec_df['Overcooked!'].sort_values()[1:11]

name
Motorsport Manager                0.330342
Drug Dealer Simulator             0.381349
Beat Cop                          0.382262
911 Operator                      0.382716
Farming Simulator 17              0.382794
Bridge Constructor Portal         0.383846
Pure Farming 2018                 0.383927
This is the Police 2              0.384520
Diaries of a Spaceport Janitor    0.384647
Farming Simulator 19              0.384929
Name: Overcooked!, dtype: float64

# Save recommender as csv file

In [10]:
rec_df.to_csv('data/recommender.csv')